In [ ]:
!pip install datasets evaluate sentencepiece sacrebleu pandas

In [ ]:
import os
import json
import time
import pandas as pd
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI
import re
import evaluate

# Set environment variable for API key (example, remove in production)
# Create your PAT token by following instructions here:
# https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
# grab the token from the token.txt file
with open("token.txt", "r") as f:
  GITHUB_TOKEN = f.read().strip()


# Initialize the OpenAI client (replace base_url if needed)
client = OpenAI(
    base_url="https://models.inference.ai.azure.com",
    api_key=os.getenv('GITHUB_TOKEN') or os.getenv('OPENAI_API_KEY')
)

# Define models
MODEL_CONFIG = [
    ("gpt-4o-mini", "ChatGPT"),
    ("Codestral-2501", "Codestral"),
]

model_choices, model_names = zip(*MODEL_CONFIG)

# Updated EXAMPLES dictionary with all 22 tasks in chronological order
EXAMPLES = {
    "task01_code_summarization": {
        "type": "analysis",
        "snippet": """public Map<String, Integer> countWordFrequency(List<String> words) {
    Map<String, Integer> freqMap = new HashMap<>();
    for (String word : words) {
        freqMap.put(word, freqMap.getOrDefault(word, 0) + 1);
    }
    return freqMap;
}
""",
        "strategies": {
            "zero_shot": "Summarize the functionality of the following Java method:",
            "few_shot": "Example:\nInput: a Java method that counts the frequency of each character in a string.\nOutput: This method takes a string as input and returns a map where each key is a character from the string, and the value is the number of times that character appears.\n\nNow, summarize the functionality of the following Java method:"
        }
    },
    "task02_bug_fixing_off_by_one": {
        "type": "analysis",
        "snippet": """def sum_range(start, end):
    total = 0
    for i in range(start, end):
        total += i
    return total
""",
        "strategies": {
            "few_shot": "Example:\nInput: start=1, end=4 → Output: 6 (This sums 1+2+3)\nNow identify and fix the off-by-one error in the following function:",
            "chain_of_thought": "Think step-by-step to find and correct the off-by-one error in the following Python function:"
        }
    },
    "task03_bug_classification_cpp": {
        "type": "analysis",
        "snippet": """int * getArray(int size) {
    int arr[size]; // Warning: local array
    return arr;    // Bug: returning pointer to local variable
}
""",
        "strategies": {
            "chain_of_thought": "Walk through your reasoning step-by-step and classify the bug in the following C++ function:",
            "prompt_chaining": "Step 1: Identify the bug in the following C++ function.\nStep 2: Explain why it occurs and its consequences."
        }
    },
    "task04_email_validator": {
        "type": "analysis",
        "snippet": """def is_valid_email(email):
    # TODO: Complete using regex
    pass
""",
        "strategies": {
            "prompt_chaining": "Step 1: Provide a regex pattern for basic email addresses (e.g., user@domain.com).\nStep 2: Implement `is_valid_email` using that pattern in the following code:",
            "chain_of_thought": "Think step-by-step to derive a regex and implement `is_valid_email` in the following code:"
        }
    },
    "task05_flask_api": {
        "type": "generation",
        "expected": """from flask import Flask, jsonify
app = Flask(__name__)

@app.route('/greet/<username>')
def greet(username):
    return jsonify({'greeting': f'Hello, {username}!'})
""",
        "strategies": {
            "zero_shot": "Create a Flask `/greet/<username>` endpoint that returns a JSON greeting.",
            "self_consistency": "Create a Flask `/greet/<username>` endpoint that returns a JSON greeting."
        }
    },
    "task06_sql_schema": {
        "type": "generation",
        "expected": """-- users(id INT PRIMARY KEY, name VARCHAR(100));
-- books(id INT PRIMARY KEY, title VARCHAR(200));
-- reviews(id INT PRIMARY KEY, user_id INT REFERENCES users(id), book_id INT REFERENCES books(id), rating INT);
""",
        "strategies": {
            "zero_shot": "Write SQL schema (tables, primary & foreign keys) for a review app with users, books, and reviews.",
            "self_consistency": "Write the SQL schema for a review app."
        }
    },
    "task07_null_deref_java": {
        "type": "analysis",
        "snippet": """public int getLength(String s) {
    return s.length(); // What if s is null?
}
""",
        "strategies": {
            "few_shot": "Example:\nInput: a method with a null check before length().\nOutput: Checks if the string is null before calling length to avoid null-dereference.\nNow identify null-dereference risk in the following code:",
            "chain_of_thought": "Reason step-by-step to find any null-dereference risk in the following Java code:"
        }
    },
    "task08_csv_parser": {
        "type": "analysis",
        "snippet": """def parse_csv_line(line):
    return line.split(',')  # Incomplete: doesn't handle quoted fields
""",
        "strategies": {
            "few_shot": "Example:\nInput: '\"a,b\",c' → Output: ['a,b','c']\nNow improve the following CSV parser:",
            "chain_of_thought": "Think step-by-step to split CSV fields correctly, handling quotes, in the following code:"
        }
    },
    "task09_kotlin_api": {
        "type": "analysis",
        "snippet": """data class Product(val id: Int, val name: String, val price: Double)
// TODO: Create GET and POST endpoints using Ktor
""",
        "strategies": {
            "prompt_chaining": "Step 1: Define GET `/product/{id}` returning JSON.\nStep 2: Define POST endpoint accepting a Product in the body for the following code:",
            "self_consistency": "Convert the following Kotlin data class to a REST API with GET and POST endpoints using Ktor."
        }
    },
    "task10_func_summary_py": {
        "type": "analysis",
        "snippet": """def reverse_words(sentence):
    return ' '.join(sentence.split()[::-1])
""",
        "strategies": {
            "prompt_chaining": "Step 1: Describe what the following function does.\nStep 2: Summarize it in one sentence:",
            "self_consistency": "Write a brief summary of the following function."
        }
    },
    "task11_prompt_from_comments": {
        "type": "analysis",
        "snippet": """# This function checks if a number is prime
def is_prime(n):
    if n <= 1: return False
    for i in range(2, int(n**0.5)+1):
        if n % i == 0: return False
    return True
""",
        "strategies": {
            "chain_of_thought": "Explain step-by-step how to craft a prompt to generate the following code:",
            "prompt_chaining": "Step 1: Write a system instruction that sets the task.\nStep 2: Provide the comment and ask for implementation for the following code:"
        }
    },
    "task12_optimize_loop": {
        "type": "analysis",
        "snippet": """squares = []
for i in range(1000000):
    squares.append(i * i)
""",
        "strategies": {
            "zero_shot": "Optimize the following Python loop for performance:",
            "chain_of_thought": "Explain step-by-step how you would optimize the following loop:"
        }
    },
    "task13_data_class_java": {
        "type": "analysis",
        "snippet": """public class Person {
    private String name;
    private int age;
    public Person(String name, int age) {
        this.name = name;
        this.age = age;
    }
    public String getName() { return name; }
    public int getAge() { return age; }
}
""",
        "strategies": {
            "zero_shot": "Convert the following Java class into a Java 16+ record:",
            "few_shot": "Example:\nInput: public class Point { int x; int y; public Point(int x, int y) { this.x = x; this.y = y; } }\nOutput: record Point(int x, int y) {}\n\nNow convert the following class:"
        }
    },
    "task14_unit_test_py": {
        "type": "analysis",
        "snippet": """def is_even(n):
    return n % 2 == 0
""",
        "strategies": {
            "zero_shot": "Write unit tests using `unittest` for the following function:",
            "few_shot": "Example:\nFunction: def add(a, b): return a + b\nTest: assert add(2, 3) == 5\n\nNow test the following function:"
        }
    },
    "task15_refactor_nested_if": {
        "type": "analysis",
        "snippet": """if user:
    if user.is_active:
        if user.is_admin:
            return True
""",
        "strategies": {
            "chain_of_thought": "Refactor the following nested `if` statements for clarity. Explain your reasoning:",
            "zero_shot": "Simplify the following nested `if` statements:"
        }
    },
    "task16_detect_infinite_loop": {
        "type": "analysis",
        "snippet": """i = 0
while i < 5:
    print(i)
""",
        "strategies": {
            "chain_of_thought": "Walk through the logic step-by-step to detect any infinite loop in the following code:",
            "prompt_chaining": "Step 1: Simulate what the following loop does.\nStep 2: Determine if it ever terminates:"
        }
    },
    "task17_javadoc_summary": {
        "type": "analysis",
        "snippet": """/**
 *
 */
public int factorial(int n) {
    if (n <= 1) return 1;
    return n * factorial(n - 1);
}
""",
        "strategies": {
            "zero_shot": "Generate a Javadoc summary for the following method:",
            "chain_of_thought": "First describe what the following method does, then generate a Javadoc summary:"
        }
    },
    "task18_comment_generator": {
        "type": "analysis",
        "snippet": """def factorial(n):
    if n == 0:
        return 1
    return n * factorial(n-1)
""",
        "strategies": {
            "prompt_chaining": "Step 1: Identify the purpose of each line.\nStep 2: Add comments to the following code:",
            "zero_shot": "Add comments to the following recursive factorial function:"
        }
    },
    "task19_code_translation_py2java": {
        "type": "analysis",
        "snippet": """def square(n):
    return n * n
""",
        "strategies": {
            "zero_shot": "Translate the following Python function to Java:",
            "prompt_chaining": "Step 1: Describe what the following function does.\nStep 2: Translate it into Java:"
        }
    },
    "task20_sorting_explanation": {
        "type": "generation",
        "expected": "Merge sort divides the array into halves, recursively sorts them, and merges the results.",
        "strategies": {
            "few_shot": "Example:\nInput: Bubble Sort\nOutput: Bubble sort repeatedly compares and swaps adjacent elements.\n\nNow explain Merge Sort:",
            "zero_shot": "Explain how merge sort works in simple terms:"
        }
    },
    "task21_missing_docstring": {
        "type": "analysis",
        "snippet": """def add(a, b):
    return a + b
""",
        "strategies": {
            "chain_of_thought": "Describe what the following function does step-by-step, then write a docstring:",
            "zero_shot": "Write a docstring for the following function:"
        }
    },
    "task22_variable_renaming": {
        "type": "analysis",
        "snippet": """a = 5
b = 10
print(a + b)
""",
        "strategies": {
            "few_shot": "Example:\nInput: a=5, b=10, return a+b\nOutput: num1=5, num2=10, return num1+num2\n\nNow rename variables to be more descriptive in the following code:",
            "chain_of_thought": "Reason step-by-step to rename variables for clarity in the following code:"
        }
    }
}


# Helper to build the prompt based on task type
def build_prompt(prefix, snippet=None):
    if snippet:
        lang = 'java' if 'java' in snippet else 'python' if 'def' in snippet else 'cpp'
        return f"{prefix}\n```{lang}\n{snippet}\n```"
    return prefix

# Safe chat completion with retry
def safe_chat_completion(model, prompt, max_retries=3):
    for attempt in range(1, max_retries + 1):
        try:
            resp = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=1024,
                temperature=0.7
            )
            return resp.choices[0].message.content.strip()
        except Exception as e:
            msg = str(e)
            m = re.search(r"wait (\d+) seconds", msg)
            if m:
                wait_secs = int(m.group(1)) + 1
                print(f"🔒 Rate-limit on '{model}', sleeping {wait_secs}s then retry (attempt {attempt})...")
                time.sleep(wait_secs)
                continue
            backoff = 2 ** attempt
            print(f"⚠️  Error on attempt {attempt} for '{model}': {e}")
            print(f"  → backing off {backoff}s before retry.")
            time.sleep(backoff)
    print(f"❌ All retries failed for model '{model}'.")
    return None

# Self-consistency runner using BLEU-4 similarity among outputs
def run_self_consistency(model, prompt, runs=3):
    outputs = []
    for i in range(1, runs + 1):
        print(f"  • SC run {i}/{runs} for model '{model}'...")
        out = safe_chat_completion(model, prompt)
        if out:
            outputs.append(out)
    if not outputs:
        return None, []
    if len(outputs) == 1:
        return outputs[0], outputs

    # Load BLEU metric
    bleu = evaluate.load("sacrebleu")

    # Compute average BLEU-4 score of each output against others
    avg_scores = []
    for i, cand in enumerate(outputs):
        score_sum = 0.0
        count = 0
        for j, ref in enumerate(outputs):
            if i == j:
                continue
            result = bleu.compute(predictions=[cand], references=[[ref]])
            score_sum += result["score"]
            count += 1
        avg_scores.append(score_sum / count if count else 0.0)

    # Select the output with the highest average BLEU score
    best_idx = avg_scores.index(max(avg_scores))
    return outputs[best_idx], outputs

# Worker executed in parallel
def worker(args):
    task_key, strat_key, model_id, model_name, prompt = args
    print(f"→ Starting: {task_key} | {strat_key} @ {model_name}")
    timestamp = datetime.utcnow().isoformat()
    if "self_consistency" in strat_key:
        output, variants = run_self_consistency(model_id, prompt)
    else:
        output = safe_chat_completion(model_id, prompt)
        variants = []
    print(f"✓ Completed: {task_key} | {strat_key} @ {model_name}")
    return {
        "task": task_key,
        "strategy": strat_key,
        "model": model_name,
        "model_id": model_id,
        "prompt": prompt,
        "output": output,
        "variants": variants,
        "timestamp": timestamp
    }

# Orchestrator: builds jobs, runs them, and saves outputs
def generate_all():
    jobs = []
    for task_key, info in EXAMPLES.items():
        snippet = info.get("snippet") if info["type"] == "analysis" else None
        for strat_key, prefix in info["strategies"].items():
            prompt = build_prompt(prefix, snippet)
            for model_id, model_name in MODEL_CONFIG:
                jobs.append((task_key, strat_key, model_id, model_name, prompt))
    print(f"🔎 Total jobs to run: {len(jobs)}")
    with ThreadPoolExecutor(max_workers=4) as executor:
        results = list(executor.map(worker, jobs))
    df = pd.DataFrame(results)
    df.to_csv("all_prompts_and_outputs.csv", index=False)
    df.to_json("all_prompts_and_outputs.json", orient="records", indent=2)
    print("✅ All outputs have been generated and saved.")

# Run the full pipeline
if __name__ == '__main__':
    generate_all()


🔎 Total jobs to run: 88
→ Starting: task01_code_summarization | zero_shot @ ChatGPT
→ Starting: task01_code_summarization | zero_shot @ Codestral
→ Starting: task01_code_summarization | few_shot @ ChatGPT
→ Starting: task01_code_summarization | few_shot @ Codestral
✓ Completed: task01_code_summarization | few_shot @ ChatGPT
→ Starting: task02_bug_fixing_off_by_one | few_shot @ ChatGPT
✓ Completed: task01_code_summarization | zero_shot @ ChatGPT
→ Starting: task02_bug_fixing_off_by_one | few_shot @ Codestral
✓ Completed: task01_code_summarization | few_shot @ Codestral
→ Starting: task02_bug_fixing_off_by_one | chain_of_thought @ ChatGPT
✓ Completed: task02_bug_fixing_off_by_one | few_shot @ ChatGPT
→ Starting: task02_bug_fixing_off_by_one | chain_of_thought @ Codestral
✓ Completed: task01_code_summarization | zero_shot @ Codestral
→ Starting: task03_bug_classification_cpp | chain_of_thought @ ChatGPT
✓ Completed: task02_bug_fixing_off_by_one | few_shot @ Codestral
→ Starting: task03_bu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')